<a href="https://colab.research.google.com/github/vondersam/sdgs_text_classifier/blob/master/experiments/glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, roc_auc_score, hamming_loss, accuracy_score
from keras import optimizers
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, Bidirectional
from keras.models import Model, Sequential
from keras.initializers import Constant
from keras.optimizers import Adam, RMSprop

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Conv1D, GlobalMaxPooling1D, CuDNNLSTM
from keras.layers import Embedding
from keras.models import load_model

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
base_dir = "gdrive/My Drive/fastai-v3/sdgs/"

Mounted at /content/gdrive


In [0]:
TEXT_DATA_DIR = f"{base_dir}dataset/cleanup_labelled.csv"
CROSS_FOLDS = f"{base_dir}dataset/cross_validation/"
GLOVE_DIR = f"{base_dir}embeddings/glove/glove.6B/"
EMBEDDINGS_DIR = f"{base_dir}embeddings/glove/"

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
NUM_EPOCHS = 20
BATCH_SIZE = 128
labels_index = [str(i) for i in range(1,18)]

In [11]:
# Load pretrained embeddings in an index mapping words in the embeddings set
# to their embeddings vector
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print(f"Found {len(embeddings_index)} word vectors.")

# second, prepare text samples and their labels
print('Processing text dataset')
df = pd.read_csv(TEXT_DATA_DIR)
df.labels = df.labels.str.split('|').apply(lambda x: [int(i) for i in x])

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df.text)
sequences = tokenizer.texts_to_sequences(df.text)
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df.labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Indexing word vectors.
Found 400000 word vectors.
Processing text dataset
Found 14736 unique tokens.
Shape of data tensor: (5182, 1000)
Shape of label tensor: (5182, 17)


In [12]:
results = []
arch = 'Conv1D_glorot_uniform'

# Cross-validation: split the data into a training set and a test set
for fold in os.listdir(CROSS_FOLDS):
    train_index = np.load(f"{CROSS_FOLDS}{fold}/train.npy")
    val_index = np.load(f"{CROSS_FOLDS}{fold}/val.npy")
    #test_index = np.load(f"{CROSS_FOLDS}{fold}/test.npy")

    x_train, x_val = data[train_index], data[val_index]
    y_train, y_val = labels[train_index], labels[val_index]
    
    print(fold)
    print('Preparing embedding matrix.')
    # prepare embedding matrix
    num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        # Ignore word if not in the n most common words
        if i > MAX_NUM_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    # load pre-trained word embeddings into an Embedding layer
    # note that we set trainable = False so as to keep the embeddings fixed
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix),
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    
    # 
    if arch == "Bidirectional_LSTM":
        x = Bidirectional(LSTM(25, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedded_sequences)
        x = GlobalMaxPooling1D()(x)
        x = Dense(50, activation="relu")(x)
        x = Dropout(0.1)(x)
        x = Dense(17, activation="sigmoid")(x)
        model = Model(inputs=sequence_input, outputs=x)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    # 0.179 with 10 epochs, 300 dimensions
    if arch == "convnet":
        # 1D convnet with global maxpooling
        x = Conv1D(128, 5, activation='relu')(embedded_sequences)
        x = MaxPooling1D(5)(x)
        x = Conv1D(128, 5, activation='relu')(x)
        x = MaxPooling1D(5)(x)
        x = Conv1D(128, 5, activation='relu')(x)
        x = GlobalMaxPooling1D()(x)
        x = Dense(128, activation='relu')(x)
        preds = Dense(len(labels_index), activation='sigmoid')(x)
        model = Model(sequence_input, preds)
        model.compile(loss='binary_crossentropy', 
                optimizer='rmsprop',
                metrics=['accuracy'])
        
    
    # 0.131 with 20 epochs, 300 dimensions
    if arch == "Conv1D_glorot_uniform":
        x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(embedded_sequences)
        avg_pool = GlobalAveragePooling1D()(x)
        max_pool = GlobalMaxPooling1D()(x)
        x = concatenate([avg_pool, max_pool])
        preds = Dense(len(labels_index), activation='sigmoid')(x)
        model = Model(sequence_input, preds)
        model.compile(loss='binary_crossentropy', 
                optimizer='rmsprop',
                metrics=['accuracy'])
   

        
    if arch == "convolution1d":
        #https://github.com/keras-team/keras/blob/master/examples/imdb_cnn.py
        model = Sequential()

        # we start off with an efficient embedding layer which maps
        # our vocab indices into embedding_dims dimensions
        #model.add(embedded_sequences)
        model.add(Embedding(num_words,
                    EMBEDDING_DIM,
                    input_length=MAX_SEQUENCE_LENGTH))
        model.add(Dropout(0.2))

        # we add a Convolution1D, which will learn filters
        # word group filters of size filter_length:
        model.add(Conv1D(filters,
                         kernel_size,
                         padding='valid',
                         activation='relu',
                         strides=1))
        # we use max pooling:
        model.add(GlobalMaxPooling1D())

        # We add a vanilla hidden layer:
        model.add(Dense(hidden_dims))
        model.add(Dropout(0.2))
        model.add(Activation('relu'))

        # We project onto a single unit output layer, and squash it with a sigmoid:
        model.add(len(labels_index))
        model.add(Activation('sigmoid'))


    model.fit(x_train, y_train,
            batch_size=128,
            epochs=NUM_EPOCHS,
            validation_data=(x_val, y_val))
    
    model.save(EMBEDDINGS_DIR + f"{arch}_{NUM_EPOCHS}epochs_{EMBEDDING_DIM}D_batchsize{BATCH_SIZE}_5fold-cross-val_{fold}.h5")

fold_1
Preparing embedding matrix.
Train on 4173 samples, validate on 516 samples
Epoch 1/20
4173/4173 [==============================] - 4s 875us/step - loss: 0.3347 - acc: 0.8917 - val_loss: 0.2629 - val_acc: 0.9203
Epoch 2/20
4173/4173 [==============================] - 1s 358us/step - loss: 0.2445 - acc: 0.9204 - val_loss: 0.2272 - val_acc: 0.9246
Epoch 3/20
4173/4173 [==============================] - 2s 362us/step - loss: 0.2099 - acc: 0.9279 - val_loss: 0.2023 - val_acc: 0.9335
Epoch 4/20
4173/4173 [==============================] - 2s 362us/step - loss: 0.1868 - acc: 0.9355 - val_loss: 0.1842 - val_acc: 0.9360
Epoch 5/20
4173/4173 [==============================] - 1s 359us/step - loss: 0.1705 - acc: 0.9414 - val_loss: 0.1728 - val_acc: 0.9406
Epoch 6/20
4173/4173 [==============================] - 1s 356us/step - loss: 0.1574 - acc: 0.9456 - val_loss: 0.1667 - val_acc: 0.9412
Epoch 7/20
4173/4173 [==============================] - 1s 359us/step - loss: 0.1471 - acc: 0.9485 - v

# Load and evaluate folds on test

In [0]:
def metrics_avg(models_testx_testy, labels_, thres=0.3):
    def calc(model, test_x, test_y):
        predictions = model.predict(test_x)>thres
        metrics = classification_report(test_y, predictions, target_names=labels_, output_dict=True)
        metrics_df = pd.DataFrame.from_dict(metrics)
        h = hamming_loss(test_y, predictions)
        roc = roc_auc_score(test_y, predictions, average='micro')
        return metrics_df, h, roc

    model_1, test_x_first, test_y_first = models_testx_testy[0]
    metrics_agg, ham, roc = calc(model_1, test_x_first, test_y_first)
    n = len(models_testx_testy)

    for model, test_x, test_y in models_testx_testy[1:]:
        metrics, h, r = calc(model, test_x, test_y)
        metrics_agg += metrics
        ham += h
        roc += r

    return metrics_agg/n, ham/n, roc/n

In [17]:
loaded_arch = 'Conv1D_glorot_uniform'
loaded_models = []
for fold in os.listdir(CROSS_FOLDS):
    print(f"Loading {fold}...")
    test_index = np.load(f"{CROSS_FOLDS}{fold}/test.npy")

    x_test = data[test_index]
    y_test = labels[test_index]
    
    load_dir = EMBEDDINGS_DIR + f"{loaded_arch}_{NUM_EPOCHS}epochs_{EMBEDDING_DIM}D_batchsize{BATCH_SIZE}_5fold-cross-val_{fold}.h5"
    loaded_model = load_model(load_dir)
    
    loaded_models.append((loaded_model, x_test, y_test))
print(f"Finished loading the {loaded_arch} models.")

Loading fold_1...
Loading fold_2...
Loading fold_3...
Loading fold_4...
Loading fold_5...
Finished loading the Conv1D_glorot_uniform models.


In [0]:
avg_results = metrics_avg(loaded_models, labels_index, thres=.3)

In [29]:
avg_results[2]

0.8148906161025143